In [1]:
# pytorch 버전: 2.5.1
import torch
import VGGNet

# torchvision 버전: 0.20.1
from torchvision import transforms
from torchvision import datasets

# 연산 장치 설정
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.cuda.manual_seed(42)
else:
    device = torch.device('cpu')
    torch.manual_seed(42)

In [ ]:
'''
image_to_tensor = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])
train_dataset = datasets.ImageFolder('ImageNet10/train', transform=image_to_tensor)

mean = torch.zeros(3)
for image, label in train_dataset:
    mean += image.mean(dim=(1,2))
mean = mean / 8000
'''

# 채널별 평균: [0.4638, 0.4585, 0.4297]

In [3]:
# 데이터 파이프라인 정의
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4638, 0.4585, 0.4297], std=[1.0, 1.0, 1.0])
])

# 데이터 로드
train_dataset = datasets.ImageFolder('ImageNet10/train', transform)
valid_dataset = datasets.ImageFolder('ImageNet10/valid', transform)

# 데이터 미니배치 처리
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=128)

In [4]:
# 모델: VGG-16
model = VGGNet.D(num_classes=10)
model.to(device)

# 옵티마이저: Momentum
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# 손실함수: Cross Entropy Loss
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    print('Epoch {:0>3}    ==>'.format(epoch+1), end='    ')

    train_loss_list = []
    train_accu_list = []
    valid_loss_list = []
    valid_accu_list = []

    # 모델을 훈련 모드로 전환
    model.train()
    for image, label in train_loader:
        image, label = image.to(device), label.to(device)
        # 누적된 그래디언트를 0으로 초기화
        optimizer.zero_grad()
        # 순전파: 전방 계산
        output = model(image)
        # 순전파: 정확도 저장
        label_pred = torch.argmax(output, dim=1)
        train_accu_list += (label==label_pred).tolist()
        # 순전파: Loss 저장
        loss = criterion(output, label)
        train_loss_list.append(loss.item())
        # 역전파: 그래디언트 계산
        loss.backward()
        # 역전파: 가중치 갱신
        optimizer.step()
    
    # 모델을 평가 모드로 전환
    model.eval()
    with torch.no_grad():
        for image, label in valid_loader:
            image, label = image.to(device), label.to(device)
            # 전방 계산
            output = model(image)
            # 정확도 저장
            label_pred = torch.argmax(output, dim=1)
            valid_accu_list += (label==label_pred).tolist()
            # Loss 저장
            loss = criterion(output, label)
            valid_loss_list.append(loss.item())
    
    # 성능지표 계산
    train_loss = sum(train_loss_list) / len(train_loss_list)
    train_accu = sum(train_accu_list) / len(train_accu_list)
    valid_loss = sum(valid_loss_list) / len(valid_loss_list)
    valid_accu = sum(valid_accu_list) / len(valid_accu_list)
    print('train_loss: {:.4f}'.format(train_loss), end='  ')
    print('train_accu: {:.4f}'.format(train_accu), end='  ')
    print('valid_loss: {:.4f}'.format(valid_loss), end='  ')
    print('valid_accu: {:.4f}'.format(valid_accu))